# TensorFlow and Neural Networks

## Introductory Tensorflow

### Jack Bennetto

#### March 7, 2017


## Objectives

* Use TensorFlow to do calculations.
* Know the advantages and disadvantages of neural networks.
* Explain the basic neural-network algorithms.
* Build a simple neural network in TensorFlow.

## TensorFlow

TensorFlow is a Google technology primarily designed to build neural networks. It's not just for neural networks – it can be used for other things as well – but that's the focus. This morning we'll learn about the basics of TensorFlow itself, and this afternoon we'll see how it's used for neural networks. Tomorrow we'll use it to build convolutional neural networks to process images.

The main object in TensorFlow is a **tensor**, which in this context is mostly just an array that may have any number of dimensions.

The first point to make is that computation in TensorFlow doesn't happen right away. First we build a **graph** that descibes the computation, and then we execute it. Let's start by building a simple graph to solve some simple linear algebra.

$$\begin{bmatrix}
3 & 3
\end{bmatrix} \cdot
\left(
\begin{bmatrix}
4 \\
5
\end{bmatrix}
+
\begin{bmatrix}
2 \\
2
\end{bmatrix}
\right)
$$

What's this equal to?

In [ ]:
import tensorflow as tf
import numpy as np
import scipy.stats as scs
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
matrix1 = tf.constant([[3., 3]])
matrix2 = tf.constant([[4.], [5]])
matrix3 = tf.constant([[2.], [2]])

total = tf.add(matrix2, matrix3)
product = tf.matmul(matrix1, total)

A **constant** is what it sounds like: an object that doesn't change. Let's look a bit at what we have. First we'll look at the types and print out the objects themselves.

In [ ]:
print type(matrix1)
print type(product)
print matrix1
print product

**Question:** What's float32? Is that the same default as a float in numpy?

Ok, so they are all tensors. (Note that at this point no computation has been done and we don't have a result yet) As in numpy we can look at the shape of a tensor; getting these wrong is a frequent source of error.

In [ ]:
print matrix1.get_shape()
print matrix2.get_shape()
print matrix3.get_shape()
print total.get_shape()
print product.get_shape()

All this checks out. Note they are actually returning special objects, not just tuples.

In [ ]:
print repr(matrix1.get_shape())

To actually do anything with that we need to create a **session**. A session is a bit like a context in Spark; all TensorFlow computation happens within it. Running a tensor within a session evaluates in as a numpy array.

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(product)

If we don't want to worry about the session object all the time, we can create an **interactive session**. That installs it as the default session so we can evaluate an object directly. We'll use an interactive session from now on, but you might not always use one.

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
product.eval()

Some nodes don't have an `eval` method. For those (if we're using an interactive session and don't want to use the session object) we use the `run` method instead.

**Practice:** Multiply

$$\begin{bmatrix}
1 \\
1
\end{bmatrix} \cdot
\begin{bmatrix}
1 & 1
\end{bmatrix}
$$


in tensorflow, without cutting-and-pasting, ideally without looking.

You can create a matrix filled with zeros (or ones) with `tf.zeros` (or `tf.ones`) that work the same as in numpy. Or you can create them with `tf.constant` from numpy arrays.

**Experiment:** Use `tf.zeros` to figure out tensorflow broadcasting rules. Are they the same as numpy?

In [ ]:
x1 = tf.zeros((1,2))
x2 = tf.zeros((1,1))
print x1.get_shape()
print x2.get_shape()
print (x1+x2).get_shape()


## Feeds and placeholders

**Placeholders** are tensors that don't have a specified value, but one that is filled in later with a **feed**. For example, suppose you want to build a graph that will multiply two numbers.

In [ ]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.mul(input1, input2)

To evaluate the output we need to provide a feed.

In [ ]:
output.eval(feed_dict={input1:6, input2:7})

Note this is the same as doing this.

In [ ]:
sess.run(output, feed_dict={input1:6, input2:7})

Note that this could take an array as well.

In [ ]:
output.eval(feed_dict={input1:[6], input2:[4,5]})

That's because we didn't specify any input dimensions. It's good to specify the input shape if we know it (in the interests of our own mental health) but we frequently leave one dimension as `None` when we're doing batch jobs so we can vary the size of the batch.

In [ ]:
input1 = tf.placeholder(tf.float32, shape=(2,))
input2 = tf.placeholder(tf.float32, shape=(None,))
output = tf.mul(input1, input2)

In [ ]:
output.eval(feed_dict={input1:[2,3], input2:[5]})

In [ ]:
output.eval(feed_dict={input1:[2,3], input2:[5,5]})

In [ ]:
output.eval(feed_dict={input1:[2,3], input2:[5,5,5]})

See why it's good to specify dimensions?

## Variables

Much of tensorflow followed a functional-programming model, but variables go around that.

Variables are tensors that might change values instead of just having a single value once evaluated. These are used (for example) to represent the strength of a connection in a network. They have an initial value; you have to run a special function (`global_variables_initializer()`) to initialize them.

## Solving an Optimization Problem

Let's solve a simple optimization problem. We want to find the minimum of a function, but it's been a few years since we've had calculus and forgot how to take a derivative.

In [ ]:
xpts = np.linspace(0, 12, 500)
ypts = np.cos(xpts)
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(xpts, ypts)

In [ ]:
x = tf.Variable(1.0, dtype=tf.float64)
y = tf.cos(x)

To solve this we create an optimizer object (there are a bunch of different types) and use that to create a minimize operation that, when run, will (incrementally) minimize its loss. Running the operation does two things, first computes the gradients of the loss with respect to the variables in the graph, and then applies the gradients to the variables.

In [ ]:
optimizer = tf.train.AdamOptimizer(1.0)
train = optimizer.minimize(y)
sess.run(tf.global_variables_initializer())
for i in xrange(800):
    if i%50 == 0:
        print "x = {0:.16f}  y = {1:.12f}".format(x.eval(), y.eval())
    train.run()